In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from zipfile import ZipFile

with ZipFile('/content/drive/My Drive/all/Face_expression_data/fer2013.zip') as zp:
  zp.extractall('/content/drive/My Drive/all/Face_expression_data')             #extract data

In [0]:
#get all the imports necessary
from __future__ import absolute_import, division, print_function, unicode_literals

try :
 %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
#from tensorflow import keras

import numpy as np
import tempfile
import pandas as pd
import os

from numpy import linalg as LA
from scipy.stats import rankdata
from shutil import copyfile, move


import h5py
import glob

from tensorflow import keras
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from keras.applications import InceptionResNetV2
from keras.applications import resnet_v2
from keras import utils
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
#read CSV
with open("/content/drive/My Drive/all/Face_expression_data/fer2013/fer2013.csv") as f:
  content = f.readlines()
  
  lines = np.array(content)
  
  num_of_instances = lines.size
  print("number of instances: ",num_of_instances)

number of instances:  35888


In [0]:
df=pd.read_csv(r'/content/drive/My Drive/all/Face_expression_data/fer2013/fer2013.csv')
df2=df[df['Usage']=='PrivateTest']
df3=df[df['Usage']=='PublicTest']
df1=df[df['Usage']=='Training']
print(df3.shape,df2.shape,df1.shape)    #get the split of data

In [0]:
p=df.iloc[1,1]
val=p.split(" ")
val=np.array(val,'float32').reshape((48,48,1))

val=tf.convert_to_tensor(val)

val=tf.image.grayscale_to_rgb(val)
pg=np.asarray(val)                                      #trial on one image 

In [0]:
y=[1]
p=utils.to_categorical(y,num_classes=3)
p=np.array(p)                                          #trial 

In [0]:
#Generate data and labels

x_train, y_train, x_test, y_test,x_val,y_val = [], [], [], [],[],[]
 
for i in range(1,num_of_instances):
 try:
  emotion, img, usage = lines[i].split(",")
  
  val = img.split(" ")
  val=np.array(val,'float32').reshape((48,48,1))
  val=tf.convert_to_tensor(val)
  val=tf.image.grayscale_to_rgb(val)
  val=np.asarray(val)
  emotion = utils.to_categorical(emotion, num_classes=7)

  if 'Training' in usage:
    y_train.append(emotion)
    x_train.append(val) 
  elif 'PrivateTest' in usage:
    y_val.append(emotion)
    x_val.append(val)
  elif 'PublicTest' in usage:
    y_test.append(emotion)
    x_test.append(val)

 except:
  print("", end="")

x_test,x_val,x_train=np.asarray(x_test),np.asarray(x_val),np.asarray(x_train)
y_test,y_val,y_train=np.asarray(y_test),np.asarray(y_val),np.asarray(y_train)

In [0]:
x_train.shape

(28709, 48, 48, 1)

In [0]:
conv_base=resnet_v2.ResNet50V2(weights=None, include_top=False,input_shape=(48,48,1))

model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(7,activation='softmax'))           #build model

In [0]:
gen = ImageDataGenerator()
print('hh')
train_generator = gen.flow(x_train, y_train, batch_size=64)
val_genr=gen.flow(x_val,y_val,batch_size=64)
test_genr=gen.flow(x_test,y_test,batch_size=64)

print('dd') 
model.compile(loss='categorical_crossentropy'
, optimizer=keras.optimizers.Adam()
, metrics=['accuracy']
)
 
history=model.fit_generator(train_generator, steps_per_epoch=128, epochs=150
                    ,validation_data=val_genr)



hh
dd
Epoch 1/150
128/128 [==============================] - 19s 152ms/step - loss: 1.8948 - accuracy: 0.2881 - val_loss: 2.1441 - val_accuracy: 0.2733
Epoch 2/150
128/128 [==============================] - 11s 86ms/step - loss: 1.7883 - accuracy: 0.3291 - val_loss: 1.8990 - val_accuracy: 0.2839
Epoch 3/150
128/128 [==============================] - 11s 86ms/step - loss: 1.7780 - accuracy: 0.3383 - val_loss: 1.5807 - val_accuracy: 0.2959
Epoch 4/150
128/128 [==============================] - 11s 87ms/step - loss: 1.7968 - accuracy: 0.3309 - val_loss: 2.1225 - val_accuracy: 0.3257
Epoch 5/150
128/128 [==============================] - 11s 87ms/step - loss: 1.7754 - accuracy: 0.3191 - val_loss: 1.8809 - val_accuracy: 0.3174
Epoch 6/150
128/128 [==============================] - 11s 87ms/step - loss: 1.7652 - accuracy: 0.3470 - val_loss: 68.3806 - val_accuracy: 0.3388
Epoch 7/150
128/128 [==============================] - 11s 87ms/step - loss: 1.7144 - accuracy: 0.3634 - val_loss: 2.1537 

In [0]:
scores=model.evaluate_generator(test_genr)
scores